<a href="https://colab.research.google.com/github/codewithdripzy/jsonformer/blob/main/Jsonformer_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate jsonformer

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

print("Loading model and tokenizer...")
model_name = "databricks/dolly-v2-3b"

offload_folder = "offload";

model = AutoModelForCausalLM.from_pretrained(model_name, use_cache=True, device_map="auto", offload_folder=offload_folder)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, use_cache=True)
print("Loaded model and tokenizer")

In [3]:
import torch
torch.cuda.empty_cache();

In [ ]:
from jsonformer.format import highlight_values
from jsonformer.main import Jsonformer
import torch # import torch for memory management

intents = {
    "type": "object",
    "properties": {
        "prompt": {
            "type": "string"
        },
        "possible_intents": {
            "type": "array",
            "items": {
                "type": "string"
            }
        },
        "predicted_intent": {
            "type": "string"
        },
        "correct_intent": {
            "type": "string"
        },
        "is_correct": {
            "type": "boolean"
        }
    }
}

prompt = """
Generate high-quality data for training a natural language understanding (NLU) model.
Each generated sample should contain the following information:
- A natural language prompt or question from a user.
- A list of possible intents the NLU model might classify, written as verbs or action phrases in verb form,
  either a single word (e.g., 'add', 'delete') or two words connected by an underscore (e.g., 'add_item', 'get_info').
- The predicted intent should also follow the same verb-based structure.
- The correct intent, which is the true action or intent the user expects.
- A boolean value that indicates whether the predicted intent matches the correct intent (true or false).

Rules:
- The intents should always be verbs (action-based), following the structure of either single words or compound words with underscores.
- Prompts should be diverse and relevant to common actions or tasks.
- Ensure that there is a mix of correct and incorrect predictions to simulate real-world model performance.

Here is the schema:
"""

builder = Jsonformer(
    model=model,
    tokenizer=tokenizer,
    json_schema=intents,
    prompt=prompt,
)

print("Generating...")
# Try offloading the model to the CPU
model.to('cpu')
# Or be more specific for the device map
# model = AutoModelForCausalLM.from_pretrained(model_name, use_cache=True, device_map={'':0}, offload_folder=offload_folder)
output = builder()
# Empty cache after generation
torch.cuda.empty_cache()

highlight_values(output)


Generating...


In [ ]:
# store the generated data

import json

# Save generated data to a JSON file
with open("generated_nlu_data_verb_intents.json", "w") as file:
    json.dump(output, file, indent=4)

print("Data saved successfully!");